Much of the code incorporated into this notebook was provied by: https://www.kaggle.com/code/jimitshah777/rnn-bi-lstm-using-keras-for-gsdc-22-starter


In [104]:
!pip install tensorflow
!pip install nb_black
!pip install nb_black > /dev/null
%load_ext lab_black


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


The system cannot find the path specified.


#### Check if GPU is available or not

In [105]:
import tensorflow as tf

tf.test.is_gpu_available()

False

#### Imports

In [106]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import plotly.express as px

pd.set_option("display.max_columns", 500)

# Convert raw data to gps

In [107]:
import glob
from dataclasses import dataclass
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from scipy.interpolate import InterpolatedUnivariateSpline

INPUT_PATH = "smartphone-decimeter-2022"

WGS84_SEMI_MAJOR_AXIS = 6378137.0  # semi-major axis of the earth's ellipsoid
WGS84_SEMI_MINOR_AXIS = 6356752.314245  # semi-minor axis of the earth's ellipsoid
WGS84_SQUARED_FIRST_ECCENTRICITY = (
    6.69437999013e-3  # measures how much the ellipsoid deviates from a perfect sphere
)
WGS84_SQUARED_SECOND_ECCENTRICITY = (
    6.73949674226e-3  # squared second eccentricity of the Earth's ellipsoid
)

HAVERSINE_RADIUS = 6_371_000  # Radius of the earth


# Earth center earth fixed coordinates
@dataclass
class ECEF:
    # x, y, z coordinates measured from earths center
    x: np.array
    y: np.array
    z: np.array

    def to_numpy(self):
        return np.stack(
            [self.x, self.y, self.z], axis=0
        )  # stack x, y, z on top of one another

    @staticmethod
    def from_numpy(pos):
        x, y, z = [np.squeeze(w) for w in np.split(pos, 3, axis=-1)]
        return ECEF(x=x, y=y, z=z)


# geodetic coordinates
@dataclass
class BLH:
    lat: np.array  # latitude
    lng: np.array  # longitude
    hgt: np.array  # geodetic height


# function converts ECEF to BLF
def ECEF_to_BLH(ecef):
    a = WGS84_SEMI_MAJOR_AXIS
    b = WGS84_SEMI_MINOR_AXIS
    e2 = WGS84_SQUARED_FIRST_ECCENTRICITY
    e2_ = WGS84_SQUARED_SECOND_ECCENTRICITY

    # cartesian coordinates
    x = ecef.x
    y = ecef.y
    z = ecef.z

    # Convert the cartesian coordinates to deggrees
    r = np.sqrt(x**2 + y**2)
    t = np.arctan2(z * (a / b), r)
    B = np.arctan2(z + (e2_ * b) * np.sin(t) ** 3, r - (e2 * a) * np.cos(t) ** 3)
    L = np.arctan2(y, x)
    n = a / np.sqrt(1 - e2 * np.sin(B) ** 2)
    H = (r / np.cos(B)) - n

    # Initialize the BLH data class and return
    return BLH(lat=B, lng=L, hgt=H)


# haversine distance is the angular sitance between two points on the surface of a sphare
def haversine_distance(blh_1, blh_2):
    dlat = blh_2.lat - blh_1.lat
    dlng = blh_2.lng - blh_1.lng
    a = (
        np.sin(dlat / 2) ** 2
        + np.cos(blh_1.lat) * np.cos(blh_2.lat) * np.sin(dlng / 2) ** 2
    )
    dist = 2 * HAVERSINE_RADIUS * np.arcsin(np.sqrt(a))
    return dist


# computer the haversine distances between two dataframe
def pandas_haversine_distance(df1, df2):
    # convert the degree measurement of latitude and longitude to radians
    blh1 = BLH(
        lat=np.deg2rad(df1["LatitudeDegrees"].to_numpy()),
        lng=np.deg2rad(df1["LongitudeDegrees"].to_numpy()),
        hgt=0,
    )
    blh2 = BLH(
        lat=np.deg2rad(df2["LatitudeDegrees"].to_numpy()),
        lng=np.deg2rad(df2["LongitudeDegrees"].to_numpy()),
        hgt=0,
    )
    return haversine_distance(
        blh1, blh2
    )  # computer haversine distance between coordinate sets


# take the gnss raw data and time scale and trpID and generate a dataframe with latitude and longitude values
def ecef_to_lat_lng(tripID, gnss_df, UnixTimeMillis):

    ecef_columns = [
        "WlsPositionXEcefMeters",
        "WlsPositionYEcefMeters",
        "WlsPositionZEcefMeters",
    ]

    columns = ["utcTimeMillis"] + ecef_columns

    ecef_df = (
        gnss_df.drop_duplicates(subset="utcTimeMillis")[
            columns
        ]  # drop duplicat rows with same time and take only specified columns
        .dropna()  # drop empty values
        .reset_index(drop=True)  # reset the idexes amd drop the old set of indexes
    )

    ecef = ECEF.from_numpy(ecef_df[ecef_columns].to_numpy())
    blh = ECEF_to_BLH(ecef)

    TIME = ecef_df[
        "utcTimeMillis"
    ].to_numpy()  # these are the time values originating from the gnss dataframe

    # Create interpolation functions
    lat = InterpolatedUnivariateSpline(TIME, blh.lat, ext=3)(UnixTimeMillis)
    lng = InterpolatedUnivariateSpline(TIME, blh.lng, ext=3)(UnixTimeMillis)

    # construct and return new latitude longitude dataframe
    return pd.DataFrame(
        {
            "tripId": tripID,  # represents both the location and phone
            "UnixTimeMillis": UnixTimeMillis,  # the internal clock time in milliseconds
            "LatitudeDegrees": np.degrees(lat),  # interpolated latitude
            "LongitudeDegrees": np.degrees(lng),  # interpolated longitude
        }
    )


# generte a distribution based on the haversine distance and then compute the average between the 50 and 95 percentile to find the
# score. (This is the metric that the competition uses)
def calc_score(tripID, pred_df, gt_df):
    d = pandas_haversine_distance(pred_df, gt_df)
    score = np.mean([np.quantile(d, 0.50), np.quantile(d, 0.95)])
    return score

In [108]:
!pip install pykalman


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [109]:
import glob
from pykalman import KalmanFilter

INPUT_PATH = "smartphone-decimeter-2022"

sample_df = pd.read_csv(f"{INPUT_PATH}/sample_submission.csv")
pred_dfs = []
for dirname in tqdm(
    sorted(glob.glob(f"{INPUT_PATH}/test/*/*"))
):  # iterate through each directory each phone in said directory
    drive, phone = dirname.split("\\")[-2:]

    tripID = f"{drive}/{phone}"  # generate the tripID for submission
    print(f"{tripID}")

    gnss_df = pd.read_csv(
        f"{dirname}/device_gnss.csv"
    )  # read the raw gnss data into a dataframe

    # retrieve he UnixTimeMillis from the sample_df where tripID matches
    UnixTimeMillis = sample_df[sample_df["tripId"] == tripID][
        "UnixTimeMillis"
    ].to_numpy()

    # Define Kalman Filter
    # kf = KalmanFilter()

    # apply kalman smoother to the interprolated latitude and longitude
    tmp = ecef_to_lat_lng(tripID, gnss_df, UnixTimeMillis)
    # smoothed_latitute_means, _ = kf.smooth(tmp["LatitudeDegrees"])
    # tmp["LatitudeDegrees"] = smoothed_latitute_means

    # smoothed_longitude_means, _ = kf.smooth(tmp["LongitudeDegrees"])
    # tmp["LongitudeDegrees"] = smoothed_longitude_means

    # add interpolated latitude and longitudes to the prediction_df
    pred_dfs.append(tmp)
sub_df = pd.concat(pred_dfs)


baselines = []
gts = []
for dirname in tqdm(sorted(glob.glob(f"{INPUT_PATH}/train/*/*"))):
    drive, phone = dirname.split("\\")[-2:]

    tripID = f"{drive}/{phone}"
    print(f"{tripID}")

    gnss_df = pd.read_csv(f"{dirname}/device_gnss.csv", low_memory=False)
    gt_df = pd.read_csv(f"{dirname}/ground_truth.csv", low_memory=False)

    # generate and append interpolatoed values using time values from ground truth
    baseline_df = ecef_to_lat_lng(tripID, gnss_df, gt_df["UnixTimeMillis"].to_numpy())
    baselines.append(baseline_df)

    # add a tripId column to the ground truth dataframe and append the dataframe to the ground truths
    gt_df["tripId"] = tripID
    gts.append(gt_df)


baselines = pd.concat(baselines)
gts = pd.concat(gts)

  0%|          | 0/36 [00:00<?, ?it/s]

2021-04-28-US-MTV-2/SamsungGalaxyS20Ultra
2021-06-22-US-MTV-1/XiaomiMi8
2021-08-12-US-MTV-1/GooglePixel4
2021-08-17-US-MTV-1/GooglePixel5
2021-08-24-US-SVL-2/GooglePixel5
2021-09-07-US-MTV-1/SamsungGalaxyS20Ultra
2021-09-14-US-MTV-1/GooglePixel5
2021-09-20-US-MTV-1/XiaomiMi8
2021-09-20-US-MTV-2/GooglePixel4
2021-09-28-US-MTV-1/GooglePixel5
2021-11-05-US-MTV-1/XiaomiMi8
2021-11-30-US-MTV-1/GooglePixel5
2022-01-04-US-MTV-1/SamsungGalaxyS20Ultra
2022-01-11-US-MTV-1/GooglePixel6Pro
2022-01-18-US-SJC-2/GooglePixel5
2022-01-26-US-MTV-1/XiaomiMi8
2022-02-01-US-SJC-1/XiaomiMi8
2022-02-08-US-SJC-1/XiaomiMi8
2022-02-15-US-SJC-1/GooglePixel5
2022-02-23-US-LAX-1/GooglePixel5
2022-02-23-US-LAX-3/XiaomiMi8
2022-02-23-US-LAX-5/XiaomiMi8
2022-02-24-US-LAX-1/SamsungGalaxyS20Ultra
2022-02-24-US-LAX-3/XiaomiMi8
2022-02-24-US-LAX-5/SamsungGalaxyS20Ultra
2022-03-14-US-MTV-1/GooglePixel5
2022-03-17-US-SJC-1/GooglePixel5
2022-03-22-US-MTV-1/SamsungGalaxyS20Ultra
2022-03-31-US-LAX-1/GooglePixel5
2022-03-31-US

  0%|          | 0/170 [00:00<?, ?it/s]

2020-05-15-US-MTV-1/GooglePixel4XL
2020-05-21-US-MTV-1/GooglePixel4
2020-05-21-US-MTV-1/GooglePixel4XL
2020-05-21-US-MTV-2/GooglePixel4
2020-05-21-US-MTV-2/GooglePixel4XL
2020-05-28-US-MTV-2/GooglePixel4
2020-05-28-US-MTV-2/GooglePixel4XL
2020-05-29-US-MTV-1/GooglePixel4
2020-05-29-US-MTV-1/GooglePixel4XL
2020-05-29-US-MTV-2/GooglePixel4
2020-05-29-US-MTV-2/GooglePixel4XL
2020-06-04-US-MTV-1/GooglePixel4
2020-06-04-US-MTV-1/GooglePixel4XL
2020-06-04-US-MTV-2/GooglePixel4
2020-06-04-US-MTV-2/GooglePixel4XL
2020-06-05-US-MTV-1/GooglePixel4
2020-06-05-US-MTV-1/GooglePixel4XL
2020-06-05-US-MTV-2/GooglePixel4
2020-06-05-US-MTV-2/GooglePixel4XL
2020-06-10-US-MTV-1/GooglePixel4
2020-06-10-US-MTV-1/GooglePixel4XL
2020-06-10-US-MTV-2/GooglePixel4
2020-06-10-US-MTV-2/GooglePixel4XL
2020-06-11-US-MTV-1/GooglePixel4
2020-06-11-US-MTV-1/GooglePixel4XL
2020-06-18-US-MTV-1/GooglePixel4
2020-06-18-US-MTV-1/GooglePixel4XL
2020-06-24-US-MTV-1/GooglePixel4
2020-06-24-US-MTV-1/GooglePixel4XL
2020-06-24-US

In [110]:
ss = pd.read_csv("smartphone-decimeter-2022/sample_submission.csv")

In [111]:
baselines["group"] = "train_baseline"
sub_df["group"] = "submission_baseline"
gts["group"] = "train_ground_truth"
combined = pd.concat([baselines, sub_df]).reset_index(drop=True).copy()

In [112]:
sub_df.head()

,tripId,UnixTimeMillis,LatitudeDegrees,LongitudeDegrees,group
0,2021-04-28-US-MTV-2/SamsungGalaxyS20Ultra,1619650832999,37.395828,-122.102968,submission_baseline
1,2021-04-28-US-MTV-2/SamsungGalaxyS20Ultra,1619650833999,37.395851,-122.102986,submission_baseline
2,2021-04-28-US-MTV-2/SamsungGalaxyS20Ultra,1619650834999,37.395835,-122.103001,submission_baseline
3,2021-04-28-US-MTV-2/SamsungGalaxyS20Ultra,1619650835999,37.395848,-122.102960,submission_baseline
4,2021-04-28-US-MTV-2/SamsungGalaxyS20Ultra,1619650836999,37.395875,-122.103001,submission_baseline


In [113]:
gts.head()

,MessageType,Provider,LatitudeDegrees,LongitudeDegrees,AltitudeMeters,SpeedMps,AccuracyMeters,BearingDegrees,UnixTimeMillis,tripId,group
0,Fix,GT,37.416619,-122.082065,NaN,0.002044,0.1,92.968750,1589573679445,2020-05-15-US-MTV-1/GooglePixel4XL,train_ground_truth
1,Fix,GT,37.416619,-122.082065,NaN,0.002198,0.1,92.969666,1589573680445,2020-05-15-US-MTV-1/GooglePixel4XL,train_ground_truth
2,Fix,GT,37.416619,-122.082065,NaN,0.001414,0.1,92.969850,1589573681445,2020-05-15-US-MTV-1/GooglePixel4XL,train_ground_truth
3,Fix,GT,37.416619,-122.082065,NaN,0.001414,0.1,92.969850,1589573682445,2020-05-15-US-MTV-1/GooglePixel4XL,train_ground_truth
4,Fix,GT,37.416619,-122.082065,NaN,0.001414,0.1,92.969910,1589573683445,2020-05-15-US-MTV-1/GooglePixel4XL,train_ground_truth


In [114]:
baselines.head()

,tripId,UnixTimeMillis,LatitudeDegrees,LongitudeDegrees,group
0,2020-05-15-US-MTV-1/GooglePixel4XL,1589573679445,37.416664,-122.082013,train_baseline
1,2020-05-15-US-MTV-1/GooglePixel4XL,1589573680445,37.416576,-122.082059,train_baseline
2,2020-05-15-US-MTV-1/GooglePixel4XL,1589573681445,37.416519,-122.082083,train_baseline
3,2020-05-15-US-MTV-1/GooglePixel4XL,1589573682445,37.416542,-122.082077,train_baseline
4,2020-05-15-US-MTV-1/GooglePixel4XL,1589573683445,37.416538,-122.082078,train_baseline


In [115]:
Lat = combined["LatitudeDegrees"].to_numpy()
Long = combined["LongitudeDegrees"].to_numpy()

In [116]:
Lat_gt = gts["LatitudeDegrees"].to_numpy()
Long_gt = gts["LongitudeDegrees"].to_numpy()

# Normalization
Bringing Data to standard form so that is makes sense in the model

In [117]:
from pandas import Series
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from math import sqrt

In [118]:
# Latitude

series_Lat_train = Series(Lat.flatten())

series_Lat_gt = Series(Lat_gt.flatten())

values_train = series_Lat_train.values
values_train = values_train.reshape((len(values_train), 1))

print(values_train.shape)

values_gt = series_Lat_gt.values
values_gt = values_gt.reshape((len(values_gt), 1))

# train the normalization
scaler_lat = MinMaxScaler()
scaler_lat = scaler_lat.fit(values_train)

# normalize the dataset and print
standardized_lat_train = scaler_lat.transform(values_train)

standardized_lat_gt = scaler_lat.transform(values_gt)

# inverse transform and print
inversed_lat_train = scaler_lat.inverse_transform(standardized_lat_train)
inversed_lat_gt = scaler_lat.inverse_transform(standardized_lat_gt)

(361730, 1)


In [119]:
sub_df.shape

(66097, 5)

In [120]:
standardized_lat_train = standardized_lat_train.reshape(361730)
standardized_lat_train_only = standardized_lat_train[0:295633]
baselines["standardized_lat"] = standardized_lat_train_only

standardized_lat_sub = standardized_lat_train[295633:]

ss["standardized_lat"] = standardized_lat_sub
gts["standardized_lat"] = standardized_lat_gt.reshape(295633)

In [121]:
# Long

series_Long_train = Series(Long.flatten())

series_Long_gt = Series(Long_gt.flatten())

values_train_long = series_Long_train.values
values_train_long = values_train_long.reshape((len(values_train_long), 1))

print(values_train_long.shape)

values_gt_long = series_Long_gt.values
values_gt_long = values_gt_long.reshape((len(values_gt_long), 1))

# train the normalization
scaler_long = MinMaxScaler()
scaler_long = scaler_long.fit(values_train_long)

# normalize the dataset
standardized_long_train = scaler_long.transform(values_train_long)

standardized_long_gt = scaler_long.transform(values_gt_long)

# inverse transform
inversed_long_train = scaler_long.inverse_transform(standardized_long_train)

inversed_long_gt = scaler_long.inverse_transform(standardized_long_gt)

(361730, 1)


In [122]:
standardized_long_train = standardized_long_train.reshape(361730)
standardized_long_train_only = standardized_long_train[0:295633]
baselines["standardized_long"] = standardized_long_train_only

standardized_long_sub = standardized_long_train[295633:]
ss["standardized_long"] = standardized_long_sub

gts["standardized_long"] = standardized_long_gt.reshape(295633)

In [123]:
gts.tail()

,MessageType,Provider,LatitudeDegrees,LongitudeDegrees,AltitudeMeters,SpeedMps,AccuracyMeters,BearingDegrees,UnixTimeMillis,tripId,group,standardized_lat,standardized_long
1609,Fix,GT,37.41605,-122.08094,-26.379000,0.006399,0.1,184.65560,1640724240000,2021-12-28-US-MTV-1/XiaomiMi8,train_ground_truth,0.953603,0.049354
1610,Fix,GT,37.41605,-122.08094,-26.379001,0.005830,0.1,184.65590,1640724241000,2021-12-28-US-MTV-1/XiaomiMi8,train_ground_truth,0.953603,0.049354
1611,Fix,GT,37.41605,-122.08094,-26.379000,0.004472,0.1,184.65613,1640724242000,2021-12-28-US-MTV-1/XiaomiMi8,train_ground_truth,0.953603,0.049354
1612,Fix,GT,37.41605,-122.08094,-26.379001,0.003609,0.1,184.65622,1640724243000,2021-12-28-US-MTV-1/XiaomiMi8,train_ground_truth,0.953603,0.049354
1613,Fix,GT,37.41605,-122.08094,-26.379000,0.004469,0.1,184.65636,1640724244000,2021-12-28-US-MTV-1/XiaomiMi8,train_ground_truth,0.953603,0.049354


In [124]:
baselines.tail()

,tripId,UnixTimeMillis,LatitudeDegrees,LongitudeDegrees,group,standardized_lat,standardized_long
1609,2021-12-28-US-MTV-1/XiaomiMi8,1640724240000,37.416021,-122.080936,train_baseline,0.953600,0.049355
1610,2021-12-28-US-MTV-1/XiaomiMi8,1640724241000,37.416016,-122.080935,train_baseline,0.953599,0.049355
1611,2021-12-28-US-MTV-1/XiaomiMi8,1640724242000,37.416027,-122.080940,train_baseline,0.953600,0.049354
1612,2021-12-28-US-MTV-1/XiaomiMi8,1640724243000,37.416036,-122.080944,train_baseline,0.953601,0.049354
1613,2021-12-28-US-MTV-1/XiaomiMi8,1640724244000,37.416031,-122.080947,train_baseline,0.953601,0.049353


In [125]:
ss.tail()

,tripId,UnixTimeMillis,LatitudeDegrees,LongitudeDegrees,standardized_lat,standardized_long
66092,2022-04-25-US-OAK-2/GooglePixel4,1650927742650,37.904611,-86.481078,0.981127,0.074859
66093,2022-04-25-US-OAK-2/GooglePixel4,1650927743642,37.904611,-86.481078,0.981136,0.074815
66094,2022-04-25-US-OAK-2/GooglePixel4,1650927744651,37.904611,-86.481078,0.981119,0.074853
66095,2022-04-25-US-OAK-2/GooglePixel4,1650927745640,37.904611,-86.481078,0.981139,0.074855
66096,2022-04-25-US-OAK-2/GooglePixel4,1650927746632,37.904611,-86.481078,0.981114,0.074891


# Reshaping the Data and Padding

In [127]:
def padTrip(df, tripId, num_rows, num_cols, value=0):
    # retrieve the standardized latitude and longitude dat data from gts which matches the trip
    trip = df.loc[(df["tripId"] == tripId), ["standardized_lat", "standardized_long"]]

    # convert the dataframe to a numpy array
    trip = trip.to_numpy()

    # pad the number array based on the difference between the number of desired rows and the size of trip instance
    tripPadded = np.pad(
        trip,  # array being badded
        (
            (0, num_rows - trip.shape[0]),  # amount to padd along x axis
            (0, num_cols - trip.shape[1]),  # amount to padd along y axis
        ),
        "constant",  # padding mode
        constant_values=value,  # this is the value that will be used in padding (maybe try making random based on range)
    )

    # print(
    #     "Padded array shape:", tripPadded.shape
    # )  # Print the shape of the padded array

    return tripPadded

In [128]:
max_length = np.amax(
    baselines["tripId"].value_counts()
)  # calculate length of trip is largest number of coordinates
desired_rows = max_length  # initialize to max_length to use in padding
desired_cols = 2
count = 0
for trip in gts.tripId.unique():  # iterate through each trip id

    if count == 0:
        trainingPadded0 = padTrip(baselines, trip, desired_rows, desired_cols)
        count = 1  # reset the count

    elif count == 1:
        trainingPadded1 = padTrip(baselines, trip, desired_rows, desired_cols)
        trainingPadded = np.stack(
            (trainingPadded0, trainingPadded1)
        )  # stack the padded arrays
        count = 2

    else:
        oneTripPadded = padTrip(baselines, trip, desired_rows, desired_cols)
        trainingPadded = np.append(
            trainingPadded, [oneTripPadded], axis=0
        )  # this sit he grounds truths padded

print(trainingPadded.shape)

(170, 3362, 2)


In [129]:
max_length = np.amax(
    gts["tripId"].value_counts()
)  # calculate length of trip is largest number of coordinates
desired_rows = max_length  # initialize to max_length to use in padding
desired_cols = 2
count = 0
for trip in gts.tripId.unique():  # iterate through each trip id

    if count == 0:

        gtsPadded = padTrip(gts, trip, desired_rows, desired_cols)
        count = 1  # reset the count

    elif count == 1:

        gtsPadded1 = padTrip(gts, trip, desired_rows, desired_cols)
        gtsPadded = np.stack((gtsPadded, gtsPadded1))  # stack the padded arrays
        count = 2

    else:

        OneTripPadded = padTrip(gts, trip, desired_rows, desired_cols)
        gtsPadded = np.append(
            gtsPadded, [oneTripPadded], axis=0
        )  # this sit he grounds truths padded

print(gtsPadded.shape)

(170, 3362, 2)


In [130]:
max_length = np.amax(ss["tripId"].value_counts())
desired_rows = max_length
desired_cols = 2
count = 0
for trip in ss.tripId.unique():
    if count == 0:
        ssPadded0 = padTrip(ss, trip, desired_rows, desired_cols)
        count = 1

    elif count == 1:
        ssPadded1 = padTrip(ss, trip, desired_rows, desired_cols)
        ssPadded = np.stack((ssPadded0, ssPadded1))
        count = 2

    else:
        oneTripPadded = padTrip(ss, trip, desired_rows, desired_cols)
        ssPadded = np.append(ssPadded, [oneTripPadded], axis=0)

print(ssPadded.shape)

(36, 4514, 2)


# Training

In [131]:
from keras.models import Sequential
from keras.models import Model

from keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import RMSprop, Adam
from keras.metrics import MeanSquaredError
from keras.layers import Dense, Dropout
from keras.layers import LSTM, Masking, Bidirectional

from tensorflow.keras.layers import Input

In [132]:
number_of_features = 2  # just feeding the gnss longitude and latitude
batch_size = 2380
# time_steps = 3362

input = Input(shape=(None, number_of_features))

masking = Masking(mask_value=0.0)(input)

# Bidrectional layer
Bidirectional_1 = Bidirectional(
    LSTM(number_of_features, return_sequences=True), merge_mode="sum"
)(masking)

# Droupout Layer which drops 15%
dropout_1 = Dropout(0.1)(Bidirectional_1)
dropout_2 = Dropout(0.1)(dropout_1)

# inlcude multiple droupout layers to avoid overfitting
out = Dense(number_of_features, activation="sigmoid")(dropout_2)

model = Model(inputs=input, outputs=out)
model.compile(
    loss="MeanSquaredError",
    optimizer=Adam(learning_rate=0.0005, decay=1e-3),
    metrics=["MeanSquaredError"],
)

c:\Users\James Pringle\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\optimizers\base_optimizer.py:32: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [133]:
# fit the model to the padded training data
model.fit(
    trainingPadded,
    gtsPadded,
    epochs=600,  # try going with 600 epochs just to maximize preformance
    batch_size=128,
    verbose=1,
    shuffle=False,
    validation_split=0.1,
)

Epoch 1/600
2/2 ━━━━━━━━━━━━━━━━━━━━ 21s 8s/step - MeanSquaredError: 0.1944 - loss: 0.1472 - val_MeanSquaredError: 0.2089 - val_loss: 0.1549
Epoch 2/600
2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 5s/step - MeanSquaredError: 0.1939 - loss: 0.1463 - val_MeanSquaredError: 0.2084 - val_loss: 0.1538
Epoch 3/600
2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 5s/step - MeanSquaredError: 0.1934 - loss: 0.1453 - val_MeanSquaredError: 0.2080 - val_loss: 0.1528
Epoch 4/600
2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 5s/step - MeanSquaredError: 0.1930 - loss: 0.1446 - val_MeanSquaredError: 0.2076 - val_loss: 0.1517
Epoch 5/600
2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 6s/step - MeanSquaredError: 0.1925 - loss: 0.1436 - val_MeanSquaredError: 0.2071 - val_loss: 0.1506
Epoch 6/600
2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 5s/step - MeanSquaredError: 0.1921 - loss: 0.1429 - val_MeanSquaredError: 0.2067 - val_loss: 0.1496
Epoch 7/600
2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 5s/step - MeanSquaredError: 0.1916 - loss: 0.1420 - val_MeanSquaredError: 0.2063 - val_loss: 0.1485
Epoch 8/600
2

#### Prediction

In [135]:
result = model.predict(trainingPadded)
print(result.shape)

6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 354ms/step
(170, 3362, 2)


In [136]:
result_sub = model.predict(ssPadded)
print(ssPadded.shape)

2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 550ms/step
(36, 4514, 2)


In [137]:
count = 0
predicted_lat = []
predicted_long = []
for trip in baselines.tripId.unique():  # itertae through the trip IDs
    timestamps = (baselines["tripId"] == trip).sum()  # get the time stamps

    # iterate through the timestamps and extract the corresponding predicted longitude and latitude
    for x in range(timestamps):
        predicted_lat.append(result[count][x][0])
        predicted_long.append(result[count][x][1])
    count = count + 1

In [138]:
def inverseTransformPreds(preds):
    np_preds = np.array(preds)  # convert the predictions into numpy array
    series_np_preds = Series(np_preds.flatten())  # convert those into a series

    values_np_preds = series_np_preds.values
    values_np_preds = values_np_preds.reshape(
        (len(values_np_preds), 1)
    )  # transpose the dimensions

    return scaler_lat.inverse_transform(
        values_np_preds
    )  # reverses earlier transformations

In [139]:
# incporated predictions into baselines as new columns
baselines["predicted_lat"] = inverseTransformPreds(predicted_lat)
baselines["predicted_long"] = inverseTransformPreds(predicted_long)

In [140]:
predicted_baseline = baselines[["tripId", "predicted_lat", "predicted_long"]].copy()

predicted_baseline.rename(columns={"predicted_lat": "LatitudeDegrees"}, inplace=True)
predicted_baseline.rename(columns={"predicted_long": "LongitudeDegrees"}, inplace=True)

scores = []
for tripID in predicted_baseline["tripId"].unique():
    score = calc_score(tripID, predicted_baseline, gts)
    scores.append(score)

mean_score = np.mean(scores)
print(f"mean_score = {mean_score:.3f}")

mean_score = 11414801.840
